In [25]:
import json

import pandas as pd
from datasets import load_dataset
from evaluate import load
from dotenv import load_dotenv

load_dotenv()
dataset_name = "scifact"

In [23]:
ds = load_dataset(f"nirantk/{dataset_name}-sparse-vectors", split="train")
print(ds)

Dataset({
    features: ['spalde-v3-lexical', '_id', 'text', 'title'],
    num_rows: 5183
})


In [6]:
trec_eval = load("trec_eval")

## Example Qrels and Runs

In [8]:
qrel = {
    "query": [0],
    "q0": ["q0"],
    "docid": ["doc_1"],
    "rel": [2]
}
run = {
    "query": [0, 0],
    "q0": ["q0", "q0"],
    "docid": ["doc_2", "doc_1"],
    "rank": [0, 1],
    "score": [1.5, 1.2],
    "system": ["test", "test"]
}
results = trec_eval.compute(predictions=[run], references=[qrel])

/opt/homebrew/Caskroom/miniconda/base/envs/lightsplade/lib/python3.9/site-packages/trectools/trec_eval.py:294: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  selection = selection[~selection["rel"].isnull()].groupby("query").first().copy()


## Load reference Qrels from test.tsv

In [16]:
df = pd.read_csv(f"../{dataset_name}/qrels/test.tsv", sep="\t")
df.head()

## Convert to qrel
qrel = {
    "query": df["query-id"].tolist(),
    "q0": ["q0"] * len(df),
    "docid": df["corpus-id"].tolist(),
    "rel": df["score"].tolist()
}

## Load the sparse vectors

['4983',
 '5836',
 '7912',
 '18670',
 '19238',
 '33370',
 '36474',
 '54440',
 '70115',
 '70490',
 '72159',
 '79447',
 '87758',
 '92308',
 '92499',
 '97884',
 '102662',
 '103007',
 '104130',
 '106301',
 '116792',
 '118568',
 '120626',
 '123859',
 '140874',
 '143251',
 '152245',
 '153744',
 '159469',
 '164189',
 '164985',
 '169264',
 '175735',
 '188911',
 '195352',
 '202259',
 '207972',
 '213017',
 '219475',
 '226488',
 '236204',
 '238409',
 '243694',
 '253672',
 '263364',
 '266641',
 '275294',
 '279052',
 '285794',
 '293661',
 '301838',
 '301866',
 '306006',
 '306311',
 '308862',
 '313394',
 '313403',
 '317204',
 '323030',
 '323335',
 '327319',
 '335029',
 '341324',
 '343052',
 '344240',
 '350542',
 '356218',
 '364522',
 '365896',
 '368506',
 '371289',
 '374902',
 '380526',
 '381602',
 '393001',
 '406733',
 '409280',
 '410286',
 '418246',
 '427082',
 '427865',
 '432261',
 '435529',
 '437924',
 '439670',
 '456304',
 '457630',
 '461550',
 '463309',
 '463533',
 '464511',
 '469066',
 '47062